Library import

In [ ]:
from functions import A_preprocessing as preprocessing

In [ ]:
df = preprocessing.read_dataset("data/raw_dataset/full_data_flightdelay.csv")

In [ ]:
df.head()

In [ ]:
preprocessing.remove_null_values(df)

In [ ]:
preprocessing.delayed_flights(df)

In [ ]:
preprocessing.create_part_of_day_column(df)
preprocessing.show_heatmap_part_of_day(df)

In [ ]:
df.head()